In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F


# class Combine(nn.Module):
#     def __init__(self):
#         self.cnn:nn.Module = None
#         self.input2size

#         self.fc1

#     def forward(self, images, linear):
#         X = fc1(torch.cat(self.cnn(images), linear))

class CNN(nn.Module):
    def __init__(self):
        nn.Module.__init__(self)
        self.conv1 = nn.Conv2d(2, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, images, linear):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = torch.cat(x, linear)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
# Include project root in path

import sys
sys.path.append("../..")

# Load data

from src.common import preprocessing, datasets
data = datasets.read_gammas()